# 4.2.1 평균 제곱 오차

평균제곱오차에 대해 책에 나와있는 설명이 잘못된 줄 알았으나.. 이건 데이터 1개에 대한 설명이기 때문에 그렇다. 책에서는 1/2로 나눠주는 부분이 있는데, 보통은 N으로 나눠준 것을 알 수 있다. 아래의 t와 y를 살펴보면 사실상 한 개의 feature에 대한 라벨값이라는 걸 알 수 있고, 숫자 2로 예측했다는 소리이다.

<p align="center"><img src="imgs/4-1s.png" width=500></p>

In [17]:
import numpy as np

y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [18]:
def mean_squared_error(y, t):
    return np.sum((y-t)**2) / 2

mean_squared_error(y, t) 

0.09750000000000003

위의 예시는 2일 확률이 가장 높다고 했을 때의 mse인데, mse가 굉장히 낮은 것을 확인할 수 있다. 잘못 예측해서 7이 가장 높다고 했을 때는 어떨까? 아래 코드를 살펴보자.

In [19]:
y = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
mean_squared_error(y, t)

0.5975

MSE가 증가했다.

# 4.2.2 교차 엔트로피 오차 (Cross entropy)

<p align="center"><img src="imgs/4-2s.jpg" width=500></p>

여기에서 log는 밑이 e인 자연로그이다. 음.. 그보다는 크로스엔트로피와 MSE를 어떤 상황에서 쓰는 지를 알아두는 것이 더 유용할 듯 하다. 

* 회귀문제 (결과값이 실수형일 경우) -> MSE를 사용
* 분류문제 (결과값이 범주형일 경우) -> 크로스 앤트로피를 사용

이 책에는 이런 부분이 빠져 있어서, 나중에 헷갈릴 수 있는데 참고해두면 좋을 듯 하다. 더 나아가서 2진 분류문제일 경우, 손실함수는 binary cross entropy가 된다는 점도 알아두자. 아, 한 가지만 더 짚고 넘어가자. 공부하다 보면 sparse categorical cross entropy도 있고 그냥 categorical cross entropy도 있는데, 이건 어떤 차이일까?

* categorical cross entropy: 원-핫 인코딩으로 된 분류문제를 다룰 경우
* sparse categorical cross entropy: 일반적인 분류문제를 다룰 경우

보통 텐서플로를 공부할 때, sparse categorical cross entropy를 자주 보게 되는데 이런 이유에서 이다. 그런데 책에서는 이걸 다 아우르는 그냥 cross entropy에 대해 다루고 있긴 하다. 그래서 아래 코드를 보면 그냥 cross entropy를 계산하고 있고, 풀어야 하는 문제를 분류 문제로 굳이 제한을 두지 않았다.

In [20]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta)) # delta -> zero division error 방지 목적

In [21]:
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

cross_entropy_error(y, t)

0.510825457099338

In [22]:
y = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
cross_entropy_error(y, t)

2.302584092994546

이 역시 1개 짜리 데이터에 대한 cross entropy라는 것을 염두해두자. 데이터가 많은 경우는 다음 절에서 설명한다.

# 4.2.3 미니배치 학습

데이터가 1개가 아닌, N 개일 경우를 생각해 본 것이다. 수식을 전개하면 아래와 같이 된다.

<p align="center"><img src="imgs/4-2s.png" width=300></p>

데이터를 100개씩 뽑아서 미니배치 학습을 돌리는 경우를 생각해보자. 우선 다시 mnist 데이터를 불러와야 한다.

In [30]:
from dataset.mnist import load_mnist

(train_x, train_y), (test_x, test_y) = load_mnist(normalize=True, one_hot_label=True)
print(train_x.shape)
print(train_y.shape)

(60000, 784)
(60000, 10)


이제 이 훈련 데이터에서 무작위로 10장을 뽑아보자.

In [31]:
train_size = train_x.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print(batch_mask)

[26703 52114 21229  4824 53445 58141 41053 43505 14685 35930]


In [32]:
x_batch = train_x[batch_mask]
y_batch = train_y[batch_mask]

print(x_batch.shape)
print(y_batch.shape)

(10, 784)
(10, 10)


# 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

함수를 다시 정의해보자. 이전과 차이가 있다면, 차원이 1일 경우 억지로 2차원으로 늘려주는 부분 뿐이다.

In [33]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y+ 1e-7)) / batch_size

위는 원핫 인코딩을 적용했을 때의 경우이고, 원핫 인코딩이 아닐 경우에는 아래처럼 하면 된다.

In [74]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    
    return -np.sum(t * np.log(y[np.arange(batch_size), t]+ 1e-7)) / batch_size # 무슨 원리로 작동하는지 도저히 이해 불가..

In [76]:
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

cross_entropy_error(y, t)

2.9957302735559908

# 4.2.5 왜 손실 함수를 사용하는가?

* 경사하강법으로 손실 함수가 최소화 되는 지점을 찾기 위해서
* 정확도를 지표로 하면 -> 가중치는 대부분의 장소에서 0이 되기 때문에 적합하지 않다.

정확도를 지표로 하면, 가중치의 미묘한 변화를 잡아내기 힘들다. 마치 계단함수와 같이 일정 자극에 도달하지 않으면, 똑같은 값을 출력하는 것이다.